# Exploring Dandiset 001359: Human Patch-seq Intracellular Electrophysiology and Transcriptomics

**Disclaimer:** This notebook was generated by AI and has not been fully verified. Please use caution when interpreting the code or results, and independently validate any findings.

This notebook helps explore and analyze Dandiset [001359 version 0.250401.1603](https://dandiarchive.org/dandiset/001359/0.250401.1603):<br>
**"20250331_AIBS_Patchseq_human"** — a collection of human multimodal Patch-seq recordings with cellular metadata.

## Notebook Outline
- Project and data overview
- Required packages
- How to connect to DANDI and retrieve assets
- Loading and understanding a representative NWB file
- Visualizations of voltage clamp and current clamp data
- Example links for external visualization
- Summary and directions for further analysis

## Overview of the Dandiset

- **Name:** 20250331_AIBS_Patchseq_human
- **Description:** HMBA Lein PatchSeq upload (human) (Q1 2025)
- **Contributors:** Gonzalez, Limary; Allen Institute for Brain Science; National Institute of Mental Health; Kalmbach, Brian; Dalley, Rachel; Lein, Ed; Lee, Brian
- **License:** CC-BY-4.0
- **Techniques:** voltage clamp, current clamp, analytical technique
- **Keywords:** Patch-seq, human, multimodal
- **Protocol:** [patch-seq-recording-and-extraction-8epv51n45l1b/v3](https://www.protocols.io/view/patch-seq-recording-and-extraction-8epv51n45l1b/v3)

**Direct Dandiset Link:** [https://dandiarchive.org/dandiset/001359/0.250401.1603](https://dandiarchive.org/dandiset/001359/0.250401.1603)

## What does this notebook cover?

1. Shows how to identify and access files in the Dandiset.
2. Demonstrates programmatic download/streaming and introspection of NWB electrophysiology data.
3. Visualizes sample voltage clamp and current clamp recordings with their stimulus protocols.
4. Suggests external analysis and visualization tools.
5. Summarizes findings and discusses further analysis options.

## Required packages

(Assumed pre-installed)
- dandi
- pynwb
- h5py
- remfile
- matplotlib
- numpy

*Note:* This notebook will stream large files from the cloud. If your internet connection is slow, you may want to modify the demonstration code to download smaller subsets or cache files.

In [1]:
# Connect to DANDI and list assets in the Dandiset
from itertools import islice
from dandi.dandiapi import DandiAPIClient

client = DandiAPIClient()
dandiset = client.get_dandiset("001359", "0.250401.1603")

metadata = dandiset.get_raw_metadata()
print(f"Dandiset name: {metadata['name']}")
print(f"Dandiset URL: {metadata['url']}")

assets = dandiset.get_assets()
print("\nFirst 5 assets in the Dandiset:")
for asset in islice(assets, 5):
    print(f"- {asset.path} (ID: {asset.identifier})")

Dandiset name: 20250331_AIBS_Patchseq_human
Dandiset URL: https://dandiarchive.org/dandiset/001359/0.250401.1603

First 5 assets in the Dandiset:
- sub-1203384279/sub-1203384279_ses-1207984257_icephys.nwb (ID: 67dc935f-cbc1-4255-bfbf-b8a3b21dacf9)
- sub-1203384279/sub-1203384279_ses-1207304122_icephys.nwb (ID: 6ea1c410-f443-44ef-a471-41fdf7bc8760)
- sub-1203384279/sub-1203384279_ses-1207262032_icephys.nwb (ID: c269347a-2d4b-4b6a-8b7c-2ef303ff503d)
- sub-1203384279/sub-1203384279_ses-1207418023_icephys.nwb (ID: f9663375-c4d6-4a9a-941f-c93c8a7b333a)
- sub-1203384279/sub-1203384279_ses-1208065964_icephys.nwb (ID: e11626f5-2176-48fc-9c4e-26da02787ba3)


## Choosing a file to explore
For this demonstration, we'll use:<br>
**sub-1203384279/sub-1203384279_ses-1207262032_icephys.nwb**<br>
Asset ID: `c269347a-2d4b-4b6a-8b7c-2ef303ff503d`

**Download URL:**<br>
https://api.dandiarchive.org/api/assets/c269347a-2d4b-4b6a-8b7c-2ef303ff503d/download/

**Explore interactively on Neurosift:**<br>
[View on Neurosift](https://neurosift.app/nwb?url=https://api.dandiarchive.org/api/assets/c269347a-2d4b-4b6a-8b7c-2ef303ff503d/download/&dandisetId=001359&dandisetVersion=0.250401.1603)

## Loading the NWB file in Python
We'll use PyNWB and remfile to stream the NWB file directly from the archive (no full download needed).

> **Tip:** We recommend always reading only the data ranges you need, as large datasets are streamed over the internet and can cause notebooks to hang or slow down.

In [2]:
import pynwb
import h5py
import remfile

url = "https://api.dandiarchive.org/api/assets/c269347a-2d4b-4b6a-8b7c-2ef303ff503d/download/"

remote_file = remfile.File(url)
h5_file = h5py.File(remote_file, 'r')
io = pynwb.NWBHDF5IO(file=h5_file)
nwb = io.read()

print(f"NWB identifier: {nwb.identifier}")
print(f"Session start: {nwb.session_start_time}")
print(f"Institution: {getattr(nwb, 'institution', 'N/A')}")
# Subject and metadata (expanded)
subject = getattr(nwb, 'subject', None)
if subject:
    print(f"Subject ID: {getattr(subject, 'subject_id', 'N/A')}")
    print(f"Sex: {getattr(subject, 'sex', 'N/A')}")
    print(f"Species: {getattr(subject, 'species', 'N/A')}")
    print(f"Age: {getattr(subject, 'age', 'N/A')}")
    print(f"Genotype: {getattr(subject, 'genotype', 'N/A')}")
else:
    print("No subject metadata available.")

# File structure exploration
print("\nTop-level acquisition series:")
print(list(nwb.acquisition.keys()))
print("\nTop-level stimulus series:")
print(list(nwb.stimulus.keys()))
print("\nProcessing modules:")
print(list(nwb.processing.keys()))
print("\nSweep table columns and shape (if present):")
if hasattr(nwb, "sweep_table"):
    try:
        df = nwb.sweep_table.to_dataframe()
        print(f"Columns: {df.columns.tolist()}")
        print(f"Shape: {df.shape}")
        print("First few sweep entries:")
        print(df.head())
    except Exception as e:
        print(f"Could not access sweep table: {str(e)}")
else:
    print("Sweep table not present.")

print("\nOther available intervals:")
if hasattr(nwb, "intervals"):
    print(list(nwb.intervals.keys()))
if hasattr(nwb, "epochs"):
    print("Epochs present.")

/home/magland/miniconda3/envs/dev/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/magland/miniconda3/envs/dev/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.2.4 because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


NWB identifier: 0b7926a648abdf35aa91c67131cbd47b1d9786dd72001bd7e2b454be71ac3a6c
Session start: 2022-09-06 17:44:39.146000+00:00
Institution: Allen Institute for Brain Science
Subject ID: 1203384279
Sex: M
Species: Homo sapiens
Age: P21170.0D
Genotype: 

Top-level acquisition series:
['data_00000_AD0', 'data_00001_AD0', 'data_00002_AD0', 'data_00003_AD0', 'data_00004_AD0', 'data_00005_AD0', 'data_00006_AD0', 'data_00007_AD0', 'data_00008_AD0', 'data_00009_AD0', 'data_00010_AD0', 'data_00011_AD0', 'data_00012_AD0', 'data_00013_AD0', 'data_00014_AD0', 'data_00015_AD0', 'data_00016_AD0', 'data_00017_AD0', 'data_00018_AD0', 'data_00019_AD0', 'data_00020_AD0', 'data_00021_AD0', 'data_00022_AD0', 'data_00023_AD0', 'data_00024_AD0', 'data_00025_AD0', 'data_00026_AD0', 'data_00027_AD0', 'data_00028_AD0', 'data_00029_AD0', 'data_00030_AD0', 'data_00031_AD0', 'data_00032_AD0', 'data_00033_AD0', 'data_00034_AD0', 'data_00035_AD0', 'data_00036_AD0', 'data_00037_AD0', 'data_00038_AD0', 'data_00039_

### File structure summary

**Top-level data modules:**

| Module             | Description               |
|--------------------|--------------------------|
| acquisition        | Main recorded time series (VoltageClampSeries/CurrentClampSeries) |
| stimulus           | Command signals delivered during acquisition |
| processing         | Processed results, e.g. detected spikes      |
| intervals/epochs   | Defined time intervals, e.g. epochs          |
| electrodes/devices | Meta-information on recording configuration  |

**Programmatic exploration example:**  
The lists printed above show available acquisition, stimulus series, processing modules, and sweep table structure. Use these to guide further programmatic analysis—see the cell above for how to retrieve keys and metadata.

- Many acquisition and stimulus series are of the form `data_XXXXX_AD0` or `data_XXXXX_DA0`
- Processing modules may be empty, sparse, or contain spikes/other processed data
- Sweep table and epochs can help organize and select protocols and trials

**Transcriptomics Note:**  
This Dandiset is Patch-seq, which combines electrophysiology and single-cell transcriptomics. The e-phys data are in NWB. If cell-matched transcriptomics data is included, it may be referenced by subject ID or metadata fields within either the NWB or as associated files in DANDI (typically as .csv or .h5ad). See asset listing and consult DandiArchive for details.

## Exploring acquisition and stimulus data

We'll visualize short segments from the first available voltage clamp and current clamp recordings, alongside their respective stimulus series.

> **Warning:** The following code only loads the first 5000 samples from each series for demonstration. Real analysis often requires context about full sweep protocols.

For sweep-based navigation, see the example code below.

import matplotlib.pyplot as plt
import numpy as np

--- Voltage clamp example ---
vclamp_keys = [k for k, v in nwb.acquisition.items() if v.__class__.__name__ == "VoltageClampSeries"]
if vclamp_keys:
    vclamp_acq = nwb.acquisition[vclamp_keys[0]]
    print(f"Selected VoltageClampSeries: {vclamp_keys[0]}")
    vstim_keys = [k for k, v in nwb.stimulus.items() if v.__class__.__name__ == "VoltageClampStimulusSeries"]
    if vstim_keys:
        vclamp_stim = nwb.stimulus[vstim_keys[0]]
        print(f"Selected VoltageClampStimulusSeries: {vstim_keys[0]}")
        Nplot = 5000
        acq_data = vclamp_acq.data[:Nplot]
        stim_data = vclamp_stim.data[:Nplot]
        dt = 1.0 / vclamp_acq.rate if hasattr(vclamp_acq, "rate") else 1.0
        t = np.arange(Nplot) * dt

        fig, ax1 = plt.subplots(figsize=(10, 5))
        color = 'tab:blue'
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel(f"Current ({vclamp_acq.unit})", color=color)
        ax1.plot(t, acq_data, color=color, lw=1, label="Measured")
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()
        color2 = 'tab:red'
        ax2.set_ylabel(f"Stimulus ({vclamp_stim.unit})", color=color2)
        ax2.plot(t, stim_data, color=color2, lw=1, label="Stimulus")
        ax2.tick_params(axis='y', labelcolor=color2)
        plt.title(f"Voltage Clamp: Current and Command (first {Nplot} samples)")
        fig.tight_layout()
        plt.show()
    else:
        print("No VoltageClampStimulusSeries found to match the VoltageClampSeries.")
else:
    print("No VoltageClampSeries found in the NWB file.")

*Above: Blue = measured current, Red = command voltage (stimulus). The plot shows a snippet of both measured current and applied stimulus in voltage clamp mode.*

--- Current clamp example ---
cclamp_keys = [k for k, v in nwb.acquisition.items() if v.__class__.__name__ == "CurrentClampSeries"]
if cclamp_keys:
    cclamp_acq = nwb.acquisition[cclamp_keys[0]]
    print(f"Selected CurrentClampSeries: {cclamp_keys[0]}")
    cstim_keys = [k for k, v in nwb.stimulus.items() if v.__class__.__name__ == "CurrentClampStimulusSeries"]
    if cstim_keys:
        cclamp_stim = nwb.stimulus[cstim_keys[0]]
        print(f"Selected CurrentClampStimulusSeries: {cstim_keys[0]}")
        Nplot = 5000
        acq_data = cclamp_acq.data[:Nplot]
        stim_data = cclamp_stim.data[:Nplot]
        dt = 1.0 / cclamp_acq.rate if hasattr(cclamp_acq, "rate") else 1.0
        t = np.arange(Nplot) * dt

        fig, ax1 = plt.subplots(figsize=(10, 5))
        color = 'tab:blue'
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel(f"Voltage ({cclamp_acq.unit})", color=color)
        ax1.plot(t, acq_data, color=color, lw=1, label="Measured")
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()
        color2 = 'tab:red'
        ax2.set_ylabel(f"Injected Current ({cclamp_stim.unit})", color=color2)
        ax2.plot(t, stim_data, color=color2, lw=1, label="Stimulus")
        ax2.tick_params(axis='y', labelcolor=color2)
        plt.title(f"Current Clamp: Voltage and Injected Current (first {Nplot} samples)")
        fig.tight_layout()
        plt.show()
    else:
        print("No CurrentClampStimulusSeries found to match the CurrentClampSeries.")
else:
    print("No CurrentClampSeries found in the NWB file.")

--- Sweeps via sweep table ---
if hasattr(nwb, "sweep_table"):
    try:
        df = nwb.sweep_table.to_dataframe()
        print("\nFirst five sweeps in table, with their series and numbers:")
        print(df[["series", "sweep_number"]].head())
        # Example: show how to fetch the PatchClampSeries for a sweep
        # (for more advanced navigation)
        first_row = df.iloc[0]
        series_object = nwb.get_acquisition(first_row["series"][0]) if isinstance(first_row["series"], (list, np.ndarray)) else nwb.get_acquisition(first_row["series"])
        print(f"First PatchClampSeries from sweep 0: {series_object}")
    except Exception as e:
        print(f"Could not access sweep table: {str(e)}")
else:
    print("No sweep table present in this NWB file.")

*Above: Blue = measured voltage, Red = injected current. This plot segment illustrates active responses to pulse protocols in current clamp mode.*

## External exploration
For broader visualization, you can view this file online at [Neurosift](https://neurosift.app/nwb?url=https://api.dandiarchive.org/api/assets/c269347a-2d4b-4b6a-8b7c-2ef303ff503d/download/&dandisetId=001359&dandisetVersion=0.250401.1603).

## Summary and further analysis directions

This notebook demonstrates streaming access and initial exploration for human Patch-seq electrophysiology in DANDI Dandiset 001359.

- Data are available in standardized NWB format, with both voltage clamp and current clamp protocols and detailed subject metadata.
- Quick plotting allows basic QC and inspection. Larger-scale or protocol-specific analysis can be done by extending these code examples.
- External tools (e.g. Neurosift, Dandi API) can further assist in data navigation.

**Potential future analyses:**
- Extract statistical summaries across sweeps or epochs.
- Link physiology to transcriptomics if available.
- Analyze cell-to-cell variability or cluster by response type.
- Visualize more complete protocol epochs using the sweep_table or epoch metadata.

*Remember to validate any result from this notebook independently prior to publication or interpretation in a scientific context.*